Thanks to @andradaolteanu for her good sentiment analysis report on covid-19 using R(https://www.kaggle.com/andradaolteanu/covid-19-sentiment-analysis/notebook). This I have done the same thing using Python. Hope you find it useful.

<img src="https://i.imgur.com/FNi8CFE.png">
<center><h1>COVID-19: EDA and Text Analysis</h1></center>

# 1. Introduction

# 2. Imports

### Libraries📚

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from datetime import time,date
import nltk
import spacy
import re

Color Palette and Custom Theme🎨

In [ ]:
data = [[[5,164,192],[133,206,218],[210,167,216]],[[166,123,197],[187,28,139],[220,38,110]],]
fig = plt.figure(figsize=(5, 5))
fig.patch.set_visible(False) 
img = plt.imshow(data,interpolation='nearest')
img.set_cmap('hot')
plt.axis('off')

**### Data📂

First let's inspect the data to see what we're working with. I'm gonna keep it simple, as we'll make it more complicated soon 😁

In [ ]:
data = pd.read_csv('../input/coronavirus-2019ncov/covid-19-all.csv')
data.rename(columns={"Country":"Country/Region","State":"State/Province"},inplace=True)
tweets = pd.read_csv('../input/covid19-tweets/covid19_tweets.csv')
#Inspect Data
data.head(5)

In [ ]:
#Inspect Tweet
tweets.head(2)

# 3. What's the situation worldwide? 🌍🌎🌏

## Reported Cases in Time 📅
> **📌Note**: Cases are stable in the first quarter of the year. However, starting April they begin to rise first linearly, then almost exponentialy starting July.

In [ ]:
data[['Confirmed','Recovered','Deaths']] = data[['Confirmed','Recovered','Deaths']].fillna(0)
data_new = pd.melt(data[['Date','Confirmed','Recovered','Deaths']],id_vars=['Date'],value_vars=['Confirmed','Recovered','Deaths'],var_name='group_var',value_name='Cases')
data_new['Date'] = pd.to_datetime(data_new['Date'])
data['Date'] = pd.to_datetime(data['Date'])
dates = data['Date'].unique()
new_df = pd.DataFrame(index = pd.date_range(dates.min(), dates.max()),columns=['Confirmed','Recovered','Deaths'])
new_df[['Confirmed','Recovered','Deaths']] = new_df.apply(lambda x:data.loc[(data['Date'] == x.name),['Confirmed','Recovered','Deaths']].sum(),axis=1)
new_df = new_df.rename_axis('Date').reset_index()
data_new = pd.melt(new_df,id_vars=['Date'],value_vars=['Confirmed','Deaths','Recovered'],var_name='group_var',value_name='Cases')
data_new = data_new.sort_values(by=['Date','group_var']).reset_index(drop=True)
data_new['label'] = data_new['group_var']
new_data = data_new.pivot_table(index=['Date'], columns='group_var')
new_data.columns = new_data.columns.droplevel().rename(None)
fig,ax = plt.subplots(1,figsize=(16,8))
new_data[['Confirmed','Recovered','Deaths']].plot(ax=ax,fontsize=15)
plt.title(label='Reported Cases In Time',loc='Left',fontsize='20')
ax.set_ylabel('frequency',fontsize=20)
ax.set_ylim([0,30000000])
ax.set_xlabel('')
plt.grid()
plt.tight_layout()

## Top Countries per Case Type

> **📌Note**: US has the lead in all cases. *China* is on 18th place in this ranking. Also, it's interesting to see that top 5 countries are from ~different continents; US from North America, Brazil from South America, India from Asia, Russia from Europe and Asia and Spain from Europe.

In [ ]:
data_new = pd.melt(data[['Date','Country/Region','Confirmed','Recovered','Deaths']],id_vars=['Date','Country/Region'],value_vars=['Confirmed','Recovered','Deaths'],var_name='group_var',value_name='Cases')
data_new['Date'] = pd.to_datetime(data_new['Date'])
new_df = data[['Country/Region','Confirmed','Recovered','Deaths']].groupby(['Country/Region']).sum().reset_index()
data_new = pd.melt(new_df,id_vars=['Country/Region'],value_vars=['Confirmed','Deaths','Recovered'],var_name='group_var',value_name='Cases')
data_new = data_new.sort_values(by=['Country/Region','group_var']).reset_index(drop=True)
new_data = data_new.pivot_table(index=['Country/Region'], columns='group_var')
new_data.columns = new_data.columns.droplevel().rename(None)
fig,ax = plt.subplots(3,figsize=(16,24))
group_labels = ['Confirmed','Deaths','Recovered']
new_data.nlargest(5, ['Confirmed']).plot(y='Confirmed',ax=ax[0],kind='bar')
new_data.nlargest(5, ['Deaths']).plot(y='Deaths',ax=ax[1],kind='bar')
new_data.nlargest(5, ['Recovered']).plot(y='Recovered',ax=ax[2],kind='bar')

## Top States per Case Type

> **📌Note**: in *Recovered* category, "Recovered" state is from US. I will figure the state later in the analysis. There is also an "Unknown" state.

In [ ]:
data['Province/State'] = data['Province/State'].fillna('Unknown')
data_new = pd.melt(data[['Date','Province/State','Confirmed','Recovered','Deaths']],id_vars=['Date','Province/State'],value_vars=['Confirmed','Recovered','Deaths'],var_name='group_var',value_name='Cases')
data_new['Date'] = pd.to_datetime(data_new['Date'])
new_df = data[['Province/State','Confirmed','Recovered','Deaths']].groupby(['Province/State']).sum().reset_index()
data_new = pd.melt(new_df,id_vars=['Province/State'],value_vars=['Confirmed','Deaths','Recovered'],var_name='group_var',value_name='Cases')
data_new = data_new.sort_values(by=['Province/State','group_var']).reset_index(drop=True)
new_data = data_new.pivot_table(index=['Province/State'], columns='group_var')
new_data.columns = new_data.columns.droplevel().rename(None)
fig,ax = plt.subplots(3,figsize=(16,24))
group_labels = ['Confirmed','Deaths','Recovered']
new_data.nlargest(5, ['Confirmed']).plot(y='Confirmed',ax=ax[0],kind='bar')
new_data.nlargest(5, ['Deaths']).plot(y='Deaths',ax=ax[1],kind='bar')
new_data.nlargest(5, ['Recovered']).plot(y='Recovered',ax=ax[2],kind='bar')

# 4. Sentiment Analysis 🐥🐣

## 4.1 Text Mining 🔨🔦

### #1. Clean Corpus Function

This predefined function is going to clean the text from:
* the punctuation - `removePunctuation`
* extra white space - `stripWhitespace`
* transforms to lower case - `tolower`
* stopwords (common words that should be ignored) - `stopwords`
* numbers - `removeNumbers`

In [ ]:
def clean_corpus(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    text= text.lower()
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    text = ' '.join(filtered_sentence)
    text = "".join(filter(lambda x: not x.isdigit(), text))
    text = text.strip()
    return text

# Work in Progress... ⏳
<div class="alert alert-block alert-info">
Grams, Sentiment Analysis, Graphs, {sentimentr}, wordclouds and many more to come ^^
</div>